#  Практическое занятие: Основы GNN — от ResNet до DeepWalk и Message Passing



##  Цели занятия
1. Освоить задачу классификации узлов графа (Node Classification)
2. Научиться работать с признаками узлов без графовой структуры (ResNet-MLP)
3. Добавить структуру графа через **DeepWalk** — получить эмбеддинги узлов
4. Реализовать базовый **Message Passing** вручную — как основу GNN
5. Сравнить результаты подходов


In [ ]:
!pip install gensim node2vec torch torch_geometric --quiet

In [ ]:
import torch, torch.nn as nn, torch.nn.functional as F
import numpy as np, random
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
import networkx as nx
from gensim.models import Word2Vec
from sklearn.metrics import f1_score


# Фиксируем seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


Описание данных:

Cora — это цитатный граф научных статей
| Элемент | Что означает | Пример |
|----------|--------------|--------|
| Узел (node) | объект | научная статья |
| Ребро (edge) | связь между объектами | ссылка между статьями |
| Признаки (features) | числовое описание узла | TF-IDF слов |
| Метка (label) | категория узла | тема статьи |

В отличие от обычных таблиц, граф несёт не только признаки узлов, но и структуру связей.  
Это позволяет использовать контекст соседей при обучении.


In [ ]:
# Загружаем датасет Cora
dataset = Planetoid(root="/tmp/Cora", name="Cora")
data = dataset[0]

print(f"Количество узлов: {data.num_nodes}")
print(f"Количество признаков у нод: {data.num_features}")
print(f"Количество классов: {dataset.num_classes}")
print(f"Количество рёбер: {data.num_edges}")

In [ ]:
num_nodes, in_dim, num_classes = data.num_nodes, data.num_features, dataset.num_classes

# Базовая модель — классификатор ResNet (без графа)

Обычная нейросеть рассматривает каждый узел изолированно:

$$
\hat{y}_v = f_\theta(x_v)
$$

где $$x_v$$ — признаки узла, а $$f_\theta$$ — обычная MLP.

**Недостаток:** модель не знает, какие узлы связаны между собой.


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, dim, p_drop=0.5):
        super().__init__()
        self.lin1 = nn.Linear(dim, dim)
        self.lin2 = nn.Linear(dim, dim)
        self.drop = nn.Dropout(p_drop)
        self.bn1 = nn.BatchNorm1d(dim)
        self.bn2 = nn.BatchNorm1d(dim)

    def forward(self, x):
        h = self.bn1(x)
        h = F.relu(h)
        h = self.drop(h)
        h = self.lin1(h)
        h = self.bn2(h)
        h = F.relu(h)
        h = self.drop(h)
        h = self.lin2(h)
        # Добавляем исходный вход x (резидуальное соединение)
        # Это помогает избежать затухания градиентов и ускоряет обучение
        return x + h

# Определяем модель ResNetMLP — многослойный перцептрон с резидуальными блоками
class ResNetMLP(nn.Module):
    def __init__(self, in_dim, hidden, out_dim, blocks=2, p_drop=0.5):
        super().__init__()
        self.inp = nn.Linear(in_dim, hidden)
        self.blocks = nn.ModuleList([ResBlock(hidden, p_drop) for _ in range(blocks)])
        self.out = nn.Linear(hidden, out_dim)

    def forward(self, x):
        x = F.relu(self.inp(x))
        for blk in self.blocks:
            x = blk(x)
        return F.log_softmax(self.out(x), dim=1)


# Функция для обучения и оценки модели
def train_eval(model, x, data, epochs=200, lr=1e-2, wd=5e-4):
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    for ep in range(epochs):
        model.train()
        opt.zero_grad()
        out = model(x)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        opt.step()

    model.eval()
    with torch.no_grad():

        out = model(x)
        pred = out.argmax(dim=1)
        acc_test = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()

        # F1-мера (macro — усреднение по классам)
        f1 = f1_score(
            data.y[data.test_mask].cpu(),
            pred[data.test_mask].cpu(),
            average='macro'
        )
    return acc_test, f1



In [ ]:
model_a = ResNetMLP(in_dim=data.x.size(1), hidden=128, out_dim=dataset.num_classes)
acc_a, f1_a = train_eval(model_a, data.x, data)
print(f"Accuracy (MLP без графа): {acc_a:.4f}")
print(f"F1 (macro): {f1_a:.4f}")

Accuracy (MLP без графа): 0.5760
F1 (macro): 0.5714


## DeepWalk — добавляем структуру графа через случайные блуждания

**Идея:**  
узлы, которые часто встречаются рядом при обходах графа, должны иметь похожие эмбеддинги.

1. Для каждого узла v ∈ V:
      - Для i от 1 до num_walks:
          walk ← RandomWalk(G, start=v, length=walk_length)
          - Добавить walk в список всех обходов

2. Обучить Skip-Gram модель (например, Word2Vec) на всех полученных обходах:
      - Каждая вершина — "слово"
      - Последовательность обхода — "предложение"
      - Контекстное окно — window_size
      - Размер вектора — embedding_dim
      - Оптимизация — Skip-Gram с Negative Sampling

3. Вернуть обученные векторные представления для всех узлов V.



In [ ]:
class DeepWalk:
    def __init__(self, walk_length=40, num_walks=10, vector_size=64,
                 window_size=5, workers=2, sg=1, epochs=5, seed=42):
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.vector_size = vector_size
        self.window_size = window_size
        self.workers = workers
        self.sg = sg
        self.epochs = epochs
        self.seed = seed
        random.seed(seed)
        np.random.seed(seed)

    def _random_walk(self, G, start):
        ## TODO

        return walk

    def _generate_walks(self, G):
        ## TODO

        return walks

    def fit(self, G):
        ## TODO

    def get_embeddings(self, nodes):
        ## TODO

        return torch.tensor(emb)



In [ ]:
G = to_networkx(data, to_undirected=True)
dw = DeepWalk(vector_size=64, epochs=5)
dw.fit(G)
emb = dw.get_embeddings(range(data.num_nodes))
X_plus = torch.cat([data.x, emb], dim=1)

model_b = ResNetMLP(in_dim=X_plus.size(1), hidden=128, out_dim=dataset.num_classes)
acc_b, f1_b = train_eval(model_b, X_plus, data)

In [ ]:
print(f"Accuracy (DeepWalk + MLP): {acc_b:.4f}")
print(f"F1 (macro): {f1_b:.4f}")

##  Механика случайных обходов

### DeepWalk
- На каждом шаге выбирается **случайный сосед** текущего узла.  
- Ходы полностью **равновероятные**.  
- Фокусируется на **локальном сообществе** (узлы, близкие друг к другу).



### Node2Vec
- Вводит **смещённые (biased) обходы**, где выбор следующего узла зависит  
  от предыдущего шага и параметров `p` и `q`.

####  Параметр `p` (return parameter)
- Контролирует вероятность **возврата** к предыдущему узлу.  
- Большое `p` → меньше возвратов → обход “уходит дальше”.

####  Параметр `q` (in–out parameter)
- Определяет направление обхода:
  - `q < 1` → **BFS-подобный поиск** — исследует локальное окружение.
  - `q > 1` → **DFS-подобный поиск** — перемещается к структурно похожим, но удалённым узлам.


##  Формально (вероятность перехода)

Переход из узла `v` в `x` при предыдущем узле `t` задаётся как:

$$
P(c_{i+1} = x \mid c_i = v) =
\begin{cases}
\frac{1}{p} & \text{если } x = t \\
1 & \text{если } (x, t) \in E \\
\frac{1}{q} & \text{если } (x, t) \notin E
\end{cases}
$$





In [ ]:
from node2vec import Node2Vec as N2V

node2vec = N2V(G, dimensions=64, walk_length=40, num_walks=10, p=1.0, q=0.5, workers=2, seed=SEED)
n2v_model = node2vec.fit(window=5, min_count=1, batch_words=256, epochs=5)

Z_n2v = np.vstack([n2v_model.wv[str(i)] for i in range(num_nodes)]).astype(np.float32)
Z_n2v = torch.tensor(Z_n2v)

In [ ]:
X_plus = torch.cat([data.x, Z_n2v], dim=1)

model_b = ResNetMLP(in_dim=X_plus.size(1), hidden=128, out_dim=num_classes, blocks=2, p_drop=0.5)
acc_b2, f1_b2 = train_eval(model_b, X_plus, data, epochs=200)


In [ ]:
print(f"Accuracy (Node2Vec + MLP): {acc_b2:.4f}")
print(f"F1 (macro): {f1_b2:.4f}")

## Реализация Message Passing вручную

**Идея:**  
каждый узел обновляет свои признаки, собирая информацию от соседей.

### Формула:

$$
h_v^{(t+1)} = \text{UPDATE}\Big(h_v^{(t)}, \text{AGGREGATE}\{h_u^{(t)} : u \in \mathcal{N}(v)\}\Big)
$$

In [ ]:
def neighbor_aggregate(x, edge_index, aggr="mean"):
    """
    Агрегирует признаки соседей узлов по типу агрегации:
    aggr ∈ {"mean", "max"}.
    """
    # TODO

    return out


In [ ]:
class MyGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, aggr):
        super().__init__()
        ## TODO

    def forward(self, x, edge_index):
        ## TODO

        # Логарифм вероятностей классов
        return F.log_softmax(x, dim=1)




In [ ]:
model_c = MyGNN(data.num_features, 64, dataset.num_classes, 'mean')
opt = torch.optim.Adam(model_c.parameters(), lr=0.01, weight_decay=5e-4)


In [ ]:
for epoch in range(1, 201):
    model_c.train()
    opt.zero_grad()
    out = model_c(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    opt.step()

model_c.eval()
with torch.no_grad():
    out = model_c(data.x, data.edge_index)
    pred = out.argmax(dim=1)

    # Accuracy на тестовых узлах
    acc_c = (pred[data.test_mask] == data.y[data.test_mask]).float().mean().item()

    # F1-мера (macro) — усреднение по всем классам
    f1_c = f1_score(
        data.y[data.test_mask].cpu(),
        pred[data.test_mask].cpu(),
        average="macro"
    )

print(f"Accuracy (Message Passing): {acc_c:.4f}")
print(f"F1-мера (macro): {f1_c:.4f}")


In [ ]:
model_c = MyGNN(data.num_features, 64, dataset.num_classes, 'max')
opt = torch.optim.Adam(model_c.parameters(), lr=0.01, weight_decay=5e-4)
